In [33]:
# Import modules

import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Import our data

In [34]:
# Import questions features

file_path1 = 'df_question_features.csv'
df_question_features = pd.read_csv(file_path1)
df_question_features.drop('Unnamed: 0', axis=1, inplace=True)
df_question_features 

,%Incorrect,Heart of Algebra,Passport to Advanced Math,Problem Solving and Data Analysis,Additional Topics,MC,SA
0,0.337851,1,0,0,0,1,0
1,0.483508,1,0,0,0,1,0
2,0.549151,1,0,0,0,1,0
3,0.509471,0,1,0,0,1,0
4,0.605813,1,0,0,0,1,0
...,...,...,...,...,...,...,...
633,0.483721,0,1,0,0,0,1
634,0.037209,1,0,0,0,0,1
635,0.283721,0,0,0,1,0,1
636,0.479070,0,1,0,0,0,1


In [35]:
# Import student features

file_path1 = 'df_student_features.csv'
file_path2 = 'df_Y.csv'
df_student_features = pd.read_csv(file_path1) #(rows=students, columns=performance metric) 
df_YY = pd.read_csv(file_path2) #(rows=question, columns=student) Student answers for each question, NaN for unanswared questions

In [36]:
df_YY

,Student_ID_0,Student_ID_1,Student_ID_2,Student_ID_3,Student_ID_4,Student_ID_5,Student_ID_6,Student_ID_7,Student_ID_8,Student_ID_9,...,Student_ID_10079,Student_ID_10080,Student_ID_10081,Student_ID_10082,Student_ID_10083,Student_ID_10084,Student_ID_10085,Student_ID_10086,Student_ID_10087,Student_ID_10088
0,0.0,0.0,1.0,NaN,1.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,1.0,NaN,0.0
1,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,1.0,1.0,NaN,1.0
2,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,1.0,NaN,0.0
3,0.0,1.0,1.0,NaN,1.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0
4,1.0,1.0,1.0,NaN,1.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,1.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df_student_features

,Level 1,Level 2,Level 3,Level 4,Level 5,Word Problems,Multiple-Choice,Grid-In,One-Variable Linear Equations,Linear Functions,...,Equivalent Expressions,One-Variable Nonlinear Equations and Two-Variable Systems,Nonlinear Functions,Area and Volume,"Lines, Angles, and Triangles",Right Triangles and Trigonometry,Circles,Complex Numbers,Math_Score,English_Score
0,60.000000,65.333333,52.333333,38.0,5.666667,43.666667,55.333333,17.666667,72.333333,55.666667,...,56.666667,36.0,26.000000,6.666667,75.000000,50.0,16.666667,33.333333,493.333333,650.000000
1,62.200000,52.200000,29.200000,15.6,14.800000,31.600000,38.400000,12.400000,63.200000,63.400000,...,33.000000,34.6,25.400000,20.000000,33.333333,20.0,10.000000,40.000000,408.000000,550.000000
2,64.333333,57.333333,48.000000,25.0,14.333333,37.000000,44.333333,33.333333,27.666667,100.000000,...,65.000000,26.0,32.666667,0.000000,0.000000,50.0,16.666667,33.333333,463.333333,546.666667
3,90.000000,74.200000,55.000000,53.4,25.400000,52.600000,66.800000,27.800000,83.400000,37.500000,...,80.000000,57.0,58.600000,42.000000,50.000000,80.0,10.000000,20.000000,556.000000,702.000000
4,66.000000,58.400000,49.800000,23.0,15.200000,48.800000,52.400000,15.200000,70.200000,66.600000,...,35.000000,31.2,33.800000,4.000000,50.000000,30.0,10.000000,0.000000,476.000000,648.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10084,100.000000,63.000000,46.000000,25.0,57.000000,46.000000,51.000000,31.000000,50.000000,100.000000,...,60.000000,40.0,20.000000,40.000000,NaN,0.0,50.000000,100.000000,490.000000,540.000000
10085,80.000000,85.000000,54.000000,36.0,0.000000,52.000000,62.000000,31.000000,100.000000,100.000000,...,50.000000,38.0,25.000000,0.000000,100.000000,100.0,0.000000,0.000000,540.000000,600.000000
10086,60.000000,15.000000,25.000000,0.0,0.000000,28.000000,24.000000,0.000000,0.000000,33.000000,...,0.000000,13.0,50.000000,0.000000,0.000000,50.0,0.000000,0.000000,330.000000,410.000000
10087,33.000000,63.000000,29.000000,38.0,14.000000,42.000000,42.000000,8.000000,50.000000,100.000000,...,20.000000,20.0,0.000000,40.000000,NaN,50.0,0.000000,0.000000,420.000000,590.000000


# Data Pre-roccessing

In [38]:
df_Y = df_YY.fillna(2) # Fill in N/A values with 2 (this number is abritrary
df_R = df_YY.notna().astype(int) # Fill in 

In [39]:
df_student_features = df_student_features.fillna(-1) # Fill in N/A values with -1

In [40]:
scaler_q = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_s = MinMaxScaler()


student_features = df_student_features.values
question_features = df_question_features.values


student_features_scaled = scaler_s.fit_transform(student_features)
question_features_scaled = scaler_q.fit_transform(question_features)


#Load data
Y = df_Y.values
R = df_R.values


X1 = student_features_scaled
X2 = question_features_scaled


num_student_features = X1.shape[1]
num_question_features = X2.shape[1]
num_questions = X2.shape[0]
num_students = Y.shape[1]


print("Y", Y.shape, "R", R.shape)
print("X1", X1.shape)
print("X2", X2.shape)


print("num_student_features:", num_student_features)
print("num_question_features:", num_question_features)
print("num_questions:",   num_questions)
print("num_students:",    num_students)

Y (638, 10089) R (638, 10089)
X1 (10089, 30)
X2 (638, 7)
num_student_features: 30
num_question_features: 7
num_questions: 638
num_students: 10089


In [41]:
Y[:,0]

array([0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
       1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [42]:
R[:,0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Lets build our model

In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define function for changing model parameters quickly
def initialize_neural_network(input_shape, num_layers, num_units, output_dim):
    model = Sequential()
    
    # Input layer
    model.add(Dense(num_units, input_shape=input_shape, activation='relu'))
    
    # Hidden layers
    for _ in range(num_layers - 1):
        model.add(Dense(num_units, activation='relu'))
    
    # Output layer
    model.add(Dense(output_dim, activation='linear'))  # Linear activation so that we can have negative outputs

    return model



# Assuming you want 3 hidden layers with 100 units each and an output dimension of 100
num_layers = 3
num_units = 20
output_dim = 150

# Initialize models
# The output dimension of out two models will have to be the same so that we can take a dot product of the two in the loss step
model_student_features = initialize_neural_network((num_student_features,), num_layers, num_units, output_dim)
model_question_features = initialize_neural_network((num_question_features,), num_layers, num_units, output_dim)

In [44]:
# Define Loss
# Loss will take the output from our model and perform a dot product operation on the two output vectors
def custom_loss(X_1, X_2, Y, R, model_1, model_2):
    
    output_1 = model_1(X_1)

    output_2 = model_2(X_2)
    
    output_1_transposed = tf.transpose(output_1) # To allow for matrix multiplication
    
    result = tf.matmul(output_2, output_1_transposed)
    
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=result) # Sigmoid Binary crossentropy loss function

    # Take the transpose of the result
    masked_cross_entropy = cross_entropy * R  # Apply mask R

    J = tf.reduce_sum(masked_cross_entropy)        

    return J

In [45]:
# Cast our data as tensors

Y = tf.cast(Y, dtype=tf.float32)
R = tf.cast(R, dtype=tf.float32)
X1 = tf.cast(X1, dtype=tf.float32)
X2 = tf.cast(X2, dtype=tf.float32)

In [46]:
# Initialize Hyperparameters
iterations = 350
learning_rate = 0.0005
lambda_ = 1
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-7

# Instantiate an optimizer.
optimizer1 = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=epsilon)
optimizer2 = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=epsilon )


for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # Compute and apply Gradients for each model seperately
    
    with tf.GradientTape() as tape:
        # Compute the cost (forward pass included in cost)
        cost_value = custom_loss(X1, X2, Y, R, model_student_features, model_question_features) # compute cost
    grads1 = tape.gradient(cost_value, model_student_features.trainable_variables) # Calculate gradients
    optimizer1.apply_gradients(zip(grads1, model_student_features.trainable_variables)) # Apply gradients
    
    del tape # delete tape to insure not interference
    
    with tf.GradientTape() as tape2:
        cost_value = custom_loss(X1, X2, Y, R, model_student_features, model_question_features)
    grads2 = tape2.gradient(cost_value, model_question_features.trainable_variables)
    optimizer2.apply_gradients(zip(grads2, model_question_features.trainable_variables))

    
    # Log periodically.
    if iter % 1 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 839804.4
Training loss at iteration 1: 833110.9
Training loss at iteration 2: 826942.5
Training loss at iteration 3: 821165.0
Training loss at iteration 4: 815659.2
Training loss at iteration 5: 810337.4
Training loss at iteration 6: 805144.6
Training loss at iteration 7: 800060.1
Training loss at iteration 8: 795085.1
Training loss at iteration 9: 790235.7
Training loss at iteration 10: 785538.3
Training loss at iteration 11: 781036.2
Training loss at iteration 12: 776786.6
Training loss at iteration 13: 772847.8
Training loss at iteration 14: 769286.9
Training loss at iteration 15: 766169.4
Training loss at iteration 16: 763541.4
Training loss at iteration 17: 761419.5
Training loss at iteration 18: 759785.3
Training loss at iteration 19: 758578.1
Training loss at iteration 20: 757692.6
Training loss at iteration 21: 756987.4
Training loss at iteration 22: 756313.9
Training loss at iteration 23: 755540.6
Training loss at iteration 24: 754572.2
Training l

Training loss at iteration 203: 624994.1
Training loss at iteration 204: 624736.8
Training loss at iteration 205: 624570.0
Training loss at iteration 206: 624501.8
Training loss at iteration 207: 624555.7
Training loss at iteration 208: 624725.1
Training loss at iteration 209: 624973.6
Training loss at iteration 210: 625256.6
Training loss at iteration 211: 625524.9
Training loss at iteration 212: 625765.1
Training loss at iteration 213: 625958.6
Training loss at iteration 214: 626097.3
Training loss at iteration 215: 626169.5
Training loss at iteration 216: 626190.4
Training loss at iteration 217: 626173.1
Training loss at iteration 218: 626140.4
Training loss at iteration 219: 626112.1
Training loss at iteration 220: 626107.7
Training loss at iteration 221: 626132.1
Training loss at iteration 222: 626184.1
Training loss at iteration 223: 626273.8
Training loss at iteration 224: 626421.1
Training loss at iteration 225: 626643.8
Training loss at iteration 226: 626977.4
Training loss at

# Evaluating model performance

In [56]:
output_1 = model_student_features(X1)
output_2 = model_question_features(X2)
output_1_transposed = tf.transpose(output_1)
result = tf.matmul(output_2, output_1_transposed)
predict = tf.nn.sigmoid(result)

# Set a threshold value
threshold = 0.5

# Apply the threshold to convert values to 0 or 1
binary_array = np.where(predict >= threshold, 1, 0)

# Create a DataFrame from the binary array
df_binary = pd.DataFrame(binary_array)

# Set the index and columns of df_Y to match df_binary
df_Y.index = df_binary.index
df_Y.columns = df_binary.columns

# Count the number of matches
matches = (df_binary == df_Y).sum().sum()

excluded_count = (df_Y == 2).sum().sum()

p = matches/((num_questions*num_students) - excluded_count)

# Display the number of matches
print("Number of matches:", matches)
print("Accuracy:", p)

Number of matches: 907202
Accuracy: 0.7481424304349475


# Making Reccomendations

<a name="5.3"></a>
### 5.3 - Finding Similar Items
The neural network above produces two feature vectors, a student feature vector , and a question feature vector. The dimensions of each vector are difficult to interpret. However, if students have perform simmilary they will have similar vectors, likewise if questions have a similar level of difficulty, they will also have similar vectors. This information can be used to make recommendations. 

A similarity measure is the squared distance between the two vectors $ \mathbf{v_m^{(k)}}$ and $\mathbf{v_m^{(i)}}$ :
$$\left\Vert \mathbf{v_m^{(k)}} - \mathbf{v_m^{(i)}}  \right\Vert^2 = \sum_{l=1}^{n}(v_{m_l}^{(k)} - v_{m_l}^{(i)})^2\tag{1}$$

In [59]:
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """     
    d = np.sum(np.square(a - b))

    return d

In [167]:
def create_dist_matrix(output):
    
    output2 = np.array(output_2)
    dist_matrix = np.zeros((output2.shape[0],output2.shape[0]))
    for i in range(output2.shape[0]):
        for j in range(output2.shape[0]):
            distance = sq_dist(output2[i,:],output2[j,:])
            dist_matrix[i,j] = distance
            
    return dist_matrix

student_similarity_matrix = create_dist_matrix(output_1)
question_similarity_matrix = create_dist_matrix(output_2)

In [164]:
def find_similarities(similarity_matrix,id_):
    
    if similarity_matrix.shape[0] == 638:
        column = f'Question_{id_}'
    else:
        column = f'Student_{id_}'
    dfs = pd.DataFrame(similarity_matrix[:,id_], columns=[column])
    dfs.index = range(1, len(dfs) + 1)
    dfs = dfs.sort_values(by=[column], ascending=True)
    
    return dfs

In [168]:
student_similaritites = find_similarities(student_similarity_matrix,5)
question_similaritites = find_similarities(question_similarity_matrix,90)

In [169]:
student_similaritites

,Question_5
6,0.000000e+00
469,2.389278e-07
481,1.327307e-05
127,2.113110e-05
351,6.839908e-05
...,...
173,1.935230e+00
170,1.964635e+00
289,2.076416e+00
232,2.238847e+00


In [171]:
question_similaritites

,Question_90
91,0.000000
47,0.000030
218,0.000588
264,0.000839
159,0.000873
...,...
625,2.133002
436,2.133998
379,2.169678
441,2.293292
